In [1]:
import os
import cv2
import time
import json
import numpy as np
import threading
import queue
from datetime import datetime
from matplotlib import pyplot as plt

In [2]:
def bb_iou(boxes1, boxes2):
    x11, y11, x12, y12 = np.split(boxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(boxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

In [3]:
frames_dir = '/home/rodrigo7/Notebook/Datasets/Eucalyptus2/Frames_LR/'
video_dir = '/home/rodrigo7/Notebook/Datasets/Eucalyptus2/Videos/'
video_files = sorted(os.listdir(video_dir))
video_files = [v for v in video_files if v.endswith('.avi') and not v.endswith('_bb.avi')]
video_files = [os.path.join(video_dir, v) for v in video_files]
video_files

['/home/rodrigo7/Notebook/Datasets/Eucalyptus2/Videos/20190726.avi']

In [12]:
1920/1080 * 256

455.1111111111111

In [14]:
256 + 128 + 64

448

In [ ]:
input_height = 1080
input_width = 1920

output_height = 256
output_width = 448

for vid in range(len(video_files)):
    video_file = video_files[vid]
    print(video_file)
    video_file = os.path.join(video_dir, video_file)
    save_file = video_file.replace('Videos', 'VideosLR')
#     save_file = os.path.join(os.path.dirname(os.path.dirname(save_file)), os.path.basename(save_file))
    if os.path.exists(save_file):
        continue
    
    try:
        os.makedirs(os.path.dirname(save_file))
    except:
        pass
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(save_file, fourcc, 30.0, (output_width, output_height))
    
    cap = cv2.VideoCapture(video_file)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(n_frames):
        print(i)
        res, frame = cap.read()
        img = cv2.resize(frame, (output_width, output_height))
        out.write(img)
    out.release()
    
        

In [ ]:
def colorir(cat):
    h, w = cat.shape[:2]
    msk = np.zeros((h,w,3), dtype = 'uint8')
    msk[cat == 1] = [0,255,0]
    msk[cat == 2] = [0,0,255]
    return(msk)

In [ ]:
def get_bboxes(contours, minArea, maxArea, minDst, maxDst):
    bboxes = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        bbox = cv2.boundingRect(cnt)
        x,y,w,h = np.array(bbox)
        if area >  minArea and area < maxArea:
            if (x > minDst) and ((x+w) < maxDst):
                bboxes.append([x,y,x+w,y+h])
    return(np.array(bboxes))

In [ ]:
for video_file in video_files:    
    print(video_file)
    save_file = video_file.replace('Videos', 'Videosbb')
    video_file = video_file.replace('Videos', 'VideosLR')
    cap = cv2.VideoCapture(video_file) # load video from file
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # get video width
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #get video height

    width = 4192
    height = 3120

    new_width = int(width/4)
    new_height = int(height/4)
    
    output_height = 384
    output_width = 512    
    
    start_time = time.time()
    counter = 0


    try:
        os.makedirs(os.path.dirname(save_file))
    except:
        pass
    try:
        os.makedirs(frames_dir)
    except:
        pass
    
    json_file = save_file.replace('.avi', '.json')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(save_file, fourcc, 30.0, (new_width, new_height))


    fgbg = cv2.bgsegm.createBackgroundSubtractorMOG(50) # background subtractor
    font = cv2.FONT_HERSHEY_SIMPLEX

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # number of frames
    # total_frames = 1

    t_start = time.time() # start the time variable for time counting   
    old_plant_boxes = new_plant_boxes = tmp_plant_boxes = np.array([]) # inicialize the boxes vector
    old_stem_boxes = new_stem_boxes = tmp_stem_boxes = np.array([])
    video_history = {}
    
    # Ajustar de acordo com a variação na altura da câmera:
    adj_height = 25

    n_plantas = 1 # counter
    speed = 25 
    last_speed = speed
    tubete_lim = 300
    speed_avg = speed * np.ones(10)
    
    ini_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    p_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    h_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 2))
    v_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 5))

    for i in range(total_frames):
        print(i)
        ret, frame = cap.read()
        img = frame.copy()
        fgmask = fgbg.apply(img)
        fgmask[:,515:] = 0
        msk = fgmask.copy()
        msk[:,:10] = 0
        msk[:,515:] = 0
        msk[:10,:] = 0
        msk[255:,:] = 0
        msk = cv2.dilate(msk, ini_kernel, iterations = 5)
        msk = (msk > 0).astype('uint8')
        
        
        # Cria duas mascaras, a primeira para planta + tubete e a segunda apenas do tubete:
        fgmask = cv2.erode(fgmask, ini_kernel, iterations = 1)

        plant_msk = fgmask.copy()
        plant_msk[280:,:] = 0
        
        stem_msk = fgmask.copy()
        stem_msk[:280,:] = 0
        stem_msk[320:,:] = 0
        
        
        # Aplica um kernel vertical, para evitar que existam interrupcoes no contorno das plantas,
        # o que causaria a existencia de dois bbox:
        plant_msk = cv2.dilate(plant_msk, v_kernel, iterations = 15)
        plant_msk = cv2.dilate(plant_msk, ini_kernel, iterations = 10)
        stem_msk = cv2.dilate(stem_msk, v_kernel, iterations = 5)
        stem_msk = cv2.dilate(stem_msk, ini_kernel, iterations = 5)
        
        
        # Encontra os bbox das plantas com tubetes, que depois eh enconlhido para ficar soh a planta:
        im_tmp, contours, hierarchy = cv2.findContours(plant_msk, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        new_plant_boxes = get_bboxes(contours, 500, 500000, 10, 500)
        
        # Encontra os bbox dos tubetes, que depois sao movidos para a posicao do caule:
        im_tmp, contours, hierarchy = cv2.findContours(stem_msk, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        new_stem_boxes = get_bboxes(contours, 10, 10000, 10, 500) 
        
        if new_plant_boxes.size > 0:
            # Ordena os bbox pelo eixo x, para que o primeiro fique a direita e o ultimo a esquerda:
            new_plant_boxes = new_plant_boxes[new_plant_boxes[:,0].argsort()[::-1]]
            
            # Define um criterio para os bbox que estao apenas na primeira metade do video:
            mcrit = np.ones(len(new_plant_boxes), dtype=bool)
            mcrit[new_plant_boxes[:,0] < 10] = False
            mcrit[new_plant_boxes[:,2] > frame.shape[1]/2] = False

            # Se ja existem bbox anteriores, atualiza com os correspondentes:
            if old_plant_boxes.size > 0 :
                areas = bb_iou(new_plant_boxes, old_plant_boxes)
                ncrit = np.where(areas.max(0) > 0)
                acrit = areas.argmax(0)[ncrit]
                old_plant_boxes[ncrit] = new_plant_boxes[acrit]
                mcrit[acrit] = False
            
            # Armazena os bbox nao correspondentes, eles seram avaliados ao final e incluidos se forem validos:
            tmp_plant_boxes = new_plant_boxes[mcrit]
                
        
        if new_stem_boxes.size > 0:
            # Ordena os bbox pelo eixo x, para que o primeiro fique a direita e o ultimo a esquerda:
            new_stem_boxes = new_stem_boxes[new_stem_boxes[:,0].argsort()[::-1]]       
            
            # Define um criterio para os bbox que estao apenas na primeira metade do video:
            mcrit = np.ones(len(new_stem_boxes), dtype=bool)
            mcrit[new_stem_boxes[:,0] < 10] = False
            mcrit[new_stem_boxes[:,2] > frame.shape[1]/2] = False
            
            # Se ja existem bbox anteriores, atualiza com os correspondentes:
            if old_stem_boxes.size > 0 :
                areas = bb_iou(new_stem_boxes, old_stem_boxes)
                ncrit = np.where(areas.max(0) > 0)
                acrit = areas.argmax(0)[ncrit]        
                
                # Utiliza os bbox correspondentes para calcular a velocidade (px/frame):
                # Obs: O calculo ocorre aqui porque eh mais estavel calcular com base nos tubetes
                last_speed += np.mean(new_stem_boxes[acrit,0] - old_stem_boxes[ncrit,0])
                if last_speed > 0:
                    speed_avg = np.insert(speed_avg, 0, last_speed)[:10]
                    print("Speed:" + str(speed))
                else:
                    last_speed = speed
                speed = int(np.mean(speed_avg))
                old_stem_boxes[ncrit] = new_stem_boxes[acrit]
                mcrit[acrit] = False
            # Armazena os bbox nao correspondentes, eles seram avaliados ao final e incluidos se forem validos:
            tmp_stem_boxes = new_stem_boxes[mcrit]
                
        
        # Se for a primeira planta a ser identificada, atribui o bbox para plotar:
        if old_plant_boxes.size == 0 and old_stem_boxes.size == 0 :
            if tmp_plant_boxes.size > 0 and tmp_stem_boxes.size > 0 :
                old_stem_boxes = tmp_stem_boxes
                old_plant_boxes = tmp_plant_boxes
                tmp_stem_boxes = tmp_plant_boxes = np.array([])

            
        # Inicia o dicionario que vai salvar as coordendas dos dois bbox para cada planta:
        frame_str = {}
        if old_plant_boxes.size > 0 and old_stem_boxes.size > 0 :
            # Garante que os bbox da planta e do caule sao da mesma planta (intersectam):
            areas = bb_iou(old_plant_boxes, old_stem_boxes)
            nidx = areas.argmax(1)
            
            for idx, ix in enumerate(nidx):
                sbb = old_stem_boxes[ix].copy()
                if areas.max(1)[idx] > 0 and (sbb[0] > 10) and (frame.shape[1] - sbb[2]) > 10:
                    tubete_lim = np.min([tubete_lim, sbb[1]])
                    sbb[1] = tubete_lim + adj_height
                    # Para cada bbox da planta realiza os ajustes na posicao, plota no frame e adiciona no dicionario:
                    bb = old_plant_boxes[idx].copy()
                    bb[3] = sbb[1]
                    frame = cv2.rectangle(frame,(bb[0], bb[1]), (bb[2], bb[3]),(0,0,255), 2)
                    cv2.putText(frame, 'P{}'.format(idx + n_plantas),(int(np.mean(bb[[0,2]])), bb[1]), font, 1,(255,255,255),2,cv2.LINE_AA)
                                        
                    # Para cada bbox do caule realiza os ajustes na posicao, plota no frame e adiciona no dicionario:
                    xm = int(np.mean(sbb[[0,2]]))
                    ym = int(np.mean(sbb[[1,3]]))
                    sbb[[0,2]] = xm + np.array([-20,20])
                    sbb[[1,3]] = [sbb[1] - 20, sbb[1]]                    
                    msk = cv2.rectangle(msk,(sbb[0], sbb[1]),(sbb[2], sbb[3] + 40),2, -1)                    
                    frame = cv2.rectangle(frame,(sbb[0], sbb[1]),(sbb[2], sbb[3]),(255,0,0), 2)
                    cv2.putText(frame, 'P{}'.format(idx + n_plantas),(xm, sbb[3]), font, 1,(255,255,255),2,cv2.LINE_AA)
                    frame_str['P{}'.format(idx + n_plantas)] = {'Plant':bb.tolist(), 'Stem':sbb.tolist(), }


        # Adiciona ao dicionario do video as informacoes desse frame:
        video_history['Frame{:04d}'.format(i)] = frame_str
        
        # Nessa parte vou adicionar o outro modelo, que vai fazer a classificacao:
        # classificar(frame, planta)
        
        # Se existem novos bbox, adiciona ao conjunto que sera avaliado no proximo frame:
        if tmp_plant_boxes.size > 0 and tmp_stem_boxes.size > 0 and tmp_plant_boxes.size == tmp_stem_boxes.size:
            old_stem_boxes = np.concatenate([tmp_stem_boxes, old_stem_boxes])
            old_plant_boxes = np.concatenate([tmp_plant_boxes, old_plant_boxes])

        # Realiza a predicao da posicao de cada bbox no proximo frame, com base na velocidade instantanea:
        if old_plant_boxes.size > 0 and old_stem_boxes.size > 0 :
            old_stem_boxes += np.int0(last_speed * np.array([[1,0,1,0]]))
            old_stem_boxes = old_stem_boxes[old_stem_boxes[:,0].argsort()[::-1]]
            
            old_plant_boxes += np.int0(last_speed * np.array([[1,0,1,0]]))
            old_plant_boxes = old_plant_boxes[old_plant_boxes[:,0].argsort()[::-1]]
            
            # Remove os bbox que estarao fora da imagem, e conta como uma nova planta:
            # Obs: Esse sera o momento de enviar o controle com a classificacao da planta
            fcrit = old_stem_boxes[:,2] < frame.shape[1]
            if not fcrit.all():
                old_plant_boxes = old_plant_boxes[fcrit]
                old_stem_boxes = old_stem_boxes[fcrit]
                n_plantas += 1         
        
        
        # Estou salvando apenas os frames com planta e movimento detectados:
        t_now = time.time()
        fps = i /(t_now - t_start)
        if i % 10 == 0:
            print('Running at {} FPS'.format(fps))
        
        out.write(cv2.resize(frame, (new_width, new_height)))
       
        if last_speed > 10 and len(frame_str) >= np.max([1,len(new_stem_boxes)]):
            print(frame_str)
            
            f_name = os.path.basename(video_file[:-4]) + '_F{:04d}'.format(i)
            jpg_name = os.path.join(frames_dir, f_name + '.jpg')
            cv2.imwrite(jpg_name, img)

            png_name = os.path.join(frames_dir, f_name + '.png')
            cv2.imwrite(png_name, msk)

            png_col_name = os.path.join(frames_dir, f_name + '_col.png')
            cv2.imwrite(png_col_name, colorir(msk))


    print("Fineshed Saving Video")
    out.release()
    
    with open(json_file, 'w') as outfile:
        json.dump(video_history, outfile)


In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(colorir(msk))